In [2]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torchmetrics.classification import Accuracy
import torchmetrics

device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!pip install torchmetrics

In [5]:
import zipfile
with zipfile.ZipFile("tomato_data.zip", 'r') as zip:
  zip.extractall("tomato_data")

In [6]:
path= '/content/tomato_data'

In [41]:
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),  #Removed RandomCrop(32) as it resized the image back
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform= transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [54]:
train_path= '/content/tomato_data/data/Training_set'
test_path= '/content/tomato_data/data/Testing_set'

train_data= datasets.ImageFolder(root=train_path, transform= train_transform)
test_set= datasets.ImageFolder(root=test_path, transform= test_transform)

for (img, label) in train_data:
  print(img.shape)
  break

torch.Size([3, 128, 128])


In [55]:
train_size = int(0.85 * len(train_data))
val_size   = len(train_data) - train_size

train_set, val_set = random_split(train_data, [train_size, val_size], generator=torch.Generator().manual_seed(42))


In [56]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_set,  batch_size=64, shuffle=False)


for (img, label) in train_loader:
  print(img.shape)
  break

torch.Size([64, 3, 128, 128])


In [58]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    # Input image is 3x128x128

    self.stack= nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),  # Equivalent to nn.MaxPool2d(kernel_size=2, stride=2, padding=0)


        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier= nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=512*4*4, out_features=512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(in_features=512, out_features=10)
    )

  def forward(self, x):
    x=self.stack(x)
    x=self.classifier(x)
    return x

In [59]:
model= CNN().to(device)

In [60]:

loss_function= nn.CrossEntropyLoss()

In [66]:
optimizer= torch.optim.Adam(model.parameters(), lr= 0.001)

scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= "max", factor= 0.5, patience= 3)


In [67]:
def train(dataloader, model, loss_function, optimizer):
  model.train()
  total_loss= 0

  for batch, (image, label) in enumerate(dataloader):
    image= image.to(device)
    label= label.to(device)

    prediction= model(image)
    loss= loss_function(prediction, label)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss+= loss

  avg_loss= total_loss / len(dataloader)
  print(f"Training Average Loss: {avg_loss:.4f}")

In [68]:
val_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def validate(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    val_accuracy.reset()

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            val_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = val_accuracy.compute() * 100  # Convert to %

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")

    return accuracy


In [69]:
test_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def test(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    test_accuracy.reset()   #Reset metric state for each epoch

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            test_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = test_accuracy.compute() * 100  # Convert to %

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

    return accuracy

In [70]:
epochs = 90
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train(train_loader, model, loss_function, optimizer)
    val_acc = validate(val_loader, model, loss_function)
    scheduler.step(val_acc)
print("\n")
print("-----------------------------------------------------------------")
print("\n")
test(test_loader, model, loss_function)


Epoch 1/90
Training Average Loss: 1.7365
Validation Loss: 1.8067, Validation Accuracy: 44.34%

Epoch 2/90
Training Average Loss: 1.6705
Validation Loss: 1.8014, Validation Accuracy: 44.34%

Epoch 3/90
Training Average Loss: 1.6427
Validation Loss: 1.8280, Validation Accuracy: 44.34%

Epoch 4/90
Training Average Loss: 1.6229
Validation Loss: 1.7332, Validation Accuracy: 44.34%

Epoch 5/90
Training Average Loss: 1.6877
Validation Loss: 1.7711, Validation Accuracy: 44.60%

Epoch 6/90
Training Average Loss: 1.8430
Validation Loss: 1.7733, Validation Accuracy: 44.47%

Epoch 7/90
Training Average Loss: 1.6977
Validation Loss: 1.8688, Validation Accuracy: 44.73%

Epoch 8/90
Training Average Loss: 1.6833
Validation Loss: 1.7728, Validation Accuracy: 44.99%

Epoch 9/90
Training Average Loss: 1.6161
Validation Loss: 1.7192, Validation Accuracy: 45.38%

Epoch 10/90
Training Average Loss: 1.6447
Validation Loss: 1.7106, Validation Accuracy: 45.64%

Epoch 11/90
Training Average Loss: 1.5979
Valida

tensor(52.9412, device='cuda:0')